In [40]:
!pip install networkx

You should consider upgrading via the 'C:\Users\james\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.



     ---------------------------------------- 1.6/1.6 MB 1.2 MB/s eta 0:00:00


In [370]:
import re
import ast
import numpy as np
import math
from functools import cache
from itertools import product

def read_file_to_string(file_path):
    with open(file_path,'r') as f:
        return f.read()
    
day_17_string = read_file_to_string('day_17_input.txt')
day_17_test = read_file_to_string('day_17_test.txt')
day_17_test_2 = read_file_to_string('day_17_test_2.txt')
# day_17_test_3 = read_file_to_string('day_17_test_3.txt')
# day_17_test_4 = read_file_to_string('day_17_test_4.txt')
# day_17_test_5 = read_file_to_string('day_17_test_5.txt')

In [6]:
def parse_string(string):
    """
    Tuple to make the function cachable
    """

    day_17 = string.split('\n')
    day_17 = tuple([tuple([int(number) for number in list(string)]) for string in day_17])

    return day_17

In [371]:
test_grid = parse_string(day_17_test)
test_grid_2 = parse_string(day_17_test_2)
input_grid = parse_string(day_17_string)

In [9]:
import heapq

def neighbours(grid, point):

    #print(len(grid[point[0]]))

    neighbours_list = [
        (grid[point[0]+1][point[1]] if point[0] != len(grid)-1 else None, (point[0]+1,point[1])),
        (grid[point[0]-1][point[1]] if point[0] != 0 else None ,(point[0]-1,point[1])),
        (grid[point[0]][point[1]+1] if point[1] != len(grid[point[0]])-1 else None ,(point[0],point[1]+1)),
        (grid[point[0]][point[1]-1] if point[1] != 0 else None ,(point[0],point[1]-1))
    ]

    neighbours_list = [neighbour for neighbour in neighbours_list if neighbour[0] != None]
    heapq.heapify(neighbours_list)

    return neighbours_list

In [11]:
for neighbour, heat_loss in neighbours(test_grid, (0,0)):
    print(neighbour, heat_loss)

3 (1, 0)
4 (0, 1)


In [14]:
@cache
def shortest_path(grid, start, current_path = (), came_from=None):
    #issue - it is going aimlessly around the grid
    print(current_path)

    # can only move left/right
    if start == (len(grid)-1,len(grid[-1])-1):
        return grid[-1][-1]
    
    else:
        short_path = 10000000000
        for heat_loss, neighbour in neighbours(grid, start):

            if neighbour != came_from and neighbour not in current_path:
                #how do I accurately do direction properly??
                #we have came_from, which tells us whether we're going same dir as last time as 
                #one of the coords will need to have a difference of 2
                
                if len(current_path) >= 2:
                    # print(start, current_path[-2])
                    # print(max([neighbour[i] - current_path[-2][i] for i in range(len(current_path[-1]))]))
                    if max([neighbour[i] - current_path[-2][i] for i in range(len(current_path[-1]))]) == 3:
                        continue

                short_path = min(short_path, shortest_path(grid, neighbour, current_path=current_path+(start,),came_from=start)+heat_loss)

    return short_path

In [348]:
def dijkstra_alg(grid, start,direction=None,steps_in_direction=0):
    #need to find a way to put the 3 limit into this function. Best way is probably to track paths - but how?? 

    edges = []

    for i in range(len(grid)):
        for j in range(len(grid[0])):
            for heat_loss, neighbour in neighbours(grid, (i,j)):
                #this gives us source, destination, weight
                edges.append([(i,j), neighbour, heat_loss])

    adj = {}

    for i in range(len(grid)):
        for j in range(len(grid[0])):
            adj[(i,j)] = []

    for src, dst, weight in edges:
        adj[src].append([dst,weight])

    shortest = {} # map vertex to dist of shortest path, give path as well?

    #0 allowed here as we don't assign weight to where we start
    min_heap = [[0,(start,direction,steps_in_direction)]]

    while min_heap:
        w1, (n1,direc,steps) = heapq.heappop(min_heap)
        

        if (n1,direc,steps) in shortest:
            continue

        shortest[(n1,direc,steps)] = w1
        if n1 == (len(grid)-1,len(grid)-1):
            break


       
        for n2, w2 in adj[n1]:

            vert_change = n2[0] - n1[0]
            hor_change = n2[1] - n1[1]

            if vert_change == 1:
                new_direction = 'down'
            elif vert_change == -1:
                new_direction = 'up'
            elif hor_change == -1:
                new_direction = 'left'
            elif hor_change == 1:
                new_direction = 'right'

            #write logic for if direction = new direction
            if new_direction == 'left' and direc == 'right':
                continue

            elif new_direction == 'right' and direc == 'left':
                continue

            elif new_direction == 'up' and direc == 'down':
                continue

            elif new_direction == 'down' and direc == 'up':
                continue

            
            if new_direction == direc:
                
                new_steps = steps + 1

                if new_steps == 3:

                    #have to have a way to show "up" is also not allowed...

                    continue
            else:
                new_steps = 0
                # if (n1,direc,steps) == ((0,2),'right',0):
                #     print(adj[n1])
        
            if ((n2,new_direction,new_steps) not in shortest):
                heapq.heappush(min_heap, [w1+w2, (n2,new_direction,new_steps)])
        #print(min_heap)

        

    return shortest

In [354]:
dij_alg_input = dijkstra_alg(input_grid, (0,0),None,steps_in_direction=0)

In [356]:
for k,v in dij_alg_input.items():
    if k[0] == (len(input_grid)-1,len(input_grid[0])-1):
        print(k)
        print(v)

((140, 140), 'down', 2)
1099


## Part 2

In [375]:
def dijkstra_alg_2(grid, start,direction=None,steps_in_direction=0):
    #need to find a way to put the 3 limit into this function. Best way is probably to track paths - but how?? 

    edges = []

    for i in range(len(grid)):
        for j in range(len(grid[0])):
            for heat_loss, neighbour in neighbours(grid, (i,j)):
                #this gives us source, destination, weight
                edges.append([(i,j), neighbour, heat_loss])

    adj = {}

    for i in range(len(grid)):
        for j in range(len(grid[0])):
            adj[(i,j)] = []

    for src, dst, weight in edges:
        adj[src].append([dst,weight])

    shortest = {} # map vertex to dist of shortest path, give path as well?

    #0 allowed here as we don't assign weight to where we start
    min_heap = [[0,(start,direction,steps_in_direction)]]

    while min_heap:
        w1, (n1,direc,steps) = heapq.heappop(min_heap)
        

        if (n1,direc,steps) in shortest:
            continue

        shortest[(n1,direc,steps)] = w1
        if n1 == (len(grid)-1,len(grid[0])-1):
            break


       
        for n2, w2 in adj[n1]:

            vert_change = n2[0] - n1[0]
            hor_change = n2[1] - n1[1]

            if vert_change == 1:
                new_direction = 'down'
            elif vert_change == -1:
                new_direction = 'up'
            elif hor_change == -1:
                new_direction = 'left'
            elif hor_change == 1:
                new_direction = 'right'

            #write logic for if direction = new direction
            if new_direction == 'left' and direc == 'right':
                continue

            elif new_direction == 'right' and direc == 'left':
                continue

            elif new_direction == 'up' and direc == 'down':
                continue

            elif new_direction == 'down' and direc == 'up':
                continue

            
            if new_direction == direc:
                
                new_steps = steps + 1

                if new_steps == 10:

                    #have to have a way to show "up" is also not allowed...

                    continue
            else:
                if steps < 3:
                    continue
                new_steps = 0
                # if (n1,direc,steps) == ((0,2),'right',0):
                #     print(adj[n1])
        
            if ((n2,new_direction,new_steps) not in shortest):
                heapq.heappush(min_heap, [w1+w2, (n2,new_direction,new_steps)])
        #print(min_heap)
        

    return shortest

In [376]:
dij_alg_2_right = dijkstra_alg_2(test_grid, (0,0),direction='right',steps_in_direction=0)

In [377]:
dij_alg_2_down = dijkstra_alg_2(test_grid, (0,0),direction='down',steps_in_direction=0)

In [378]:
dij_alg_test_2_down = dijkstra_alg_2(test_grid_2, (0,0),direction='down',steps_in_direction=0)
dij_alg_test_2_right = dijkstra_alg_2(test_grid_2, (0,0),direction='right',steps_in_direction=0)

In [379]:
for k,v in dij_alg_2_right.items():
    if k[0] == (12,12):
        print(k)
        print(v)

((12, 12), 'down', 7)
94


In [380]:
for k,v in dij_alg_2_down.items():
    if k[0] == (12,12):
        print(k)
        print(v)

((12, 12), 'right', 7)
110


In [381]:
test_grid_2

((1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1),
 (9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 1),
 (9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 1),
 (9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 1),
 (9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 1))

In [382]:
for k,v in dij_alg_test_2_right.items():
    if k[0] == (4,11):
        print(k)
        print(v)

((4, 11), 'right', 1)
55


In [374]:
dij_alg_test_2_right

{((0, 0), 'right', 0): 0,
 ((0, 1), 'right', 1): 1,
 ((0, 2), 'right', 2): 2,
 ((0, 3), 'right', 3): 3,
 ((0, 4), 'right', 4): 4,
 ((0, 5), 'right', 5): 5,
 ((0, 6), 'right', 6): 6,
 ((0, 7), 'right', 7): 7,
 ((0, 8), 'right', 8): 8,
 ((0, 9), 'right', 9): 9,
 ((1, 3), 'down', 0): 12,
 ((1, 4), 'down', 0): 13,
 ((1, 5), 'down', 0): 14,
 ((1, 6), 'down', 0): 15,
 ((1, 7), 'down', 0): 16,
 ((1, 8), 'down', 0): 17,
 ((1, 9), 'down', 0): 18,
 ((2, 3), 'down', 1): 21,
 ((2, 4), 'down', 1): 22,
 ((2, 5), 'down', 1): 23,
 ((2, 6), 'down', 1): 24,
 ((2, 7), 'down', 1): 25,
 ((2, 8), 'down', 1): 26,
 ((2, 9), 'down', 1): 27,
 ((3, 3), 'down', 2): 30,
 ((3, 4), 'down', 2): 31,
 ((3, 5), 'down', 2): 32,
 ((3, 6), 'down', 2): 33,
 ((3, 7), 'down', 2): 34,
 ((3, 8), 'down', 2): 35,
 ((3, 9), 'down', 2): 36,
 ((4, 3), 'down', 3): 39,
 ((4, 4), 'down', 3): 40}

In [383]:
dij_alg_2_right = dijkstra_alg_2(input_grid, (0,0),direction='right',steps_in_direction=0)

In [384]:
dij_alg_2_down = dijkstra_alg_2(input_grid, (0,0),direction='down',steps_in_direction=0)

In [385]:
for k,v in dij_alg_2_right.items():
    if k[0] == (140, 140):
        print(k)
        print(v)

((140, 140), 'down', 9)
1266


In [386]:
for k,v in dij_alg_2_down.items():
    if k[0] == (140, 140):
        print(k)
        print(v)

((140, 140), 'down', 9)
1284
